# Title: Wrangling csv files



## Preliminaries

## Work

This is where work gets done.

#2 Create blank Excel file named BCM.xlsx

In [1]:
import pandas as pd
from pathlib import Path
from openpyxl import Workbook, load_workbook
from openpyxl.utils.dataframe import dataframe_to_rows
from openpyxl.styles import NamedStyle


# Define the path to the logs folder and the output Excel file
logs_folder = Path("data/logs")
output_file = "output/BCM.xlsx"

# Create a blank Excel workbook
wb = Workbook()
wb.save(output_file)

#3 Insert the contents of each csv into a new sheet in BCM.xlsx

In [2]:

for csv_file in logs_folder.glob("*.csv"):
    # Read the CSV file into a DataFrame
    df = pd.read_csv(csv_file, header=None, names=['datetime', 'scale', 'temperature'], parse_dates=['datetime'])
    
    # Load the existing workbook
    wb = load_workbook(output_file)
    
    # Add a new sheet with the name of the CSV file (without extension)
    sheet_name = csv_file.stem
    ws = wb.create_sheet(title=sheet_name)
    
    # Write the DataFrame to the new sheet
    for row in df.itertuples(index=False, name=None):
        ws.append(row)
    
    # Save the workbook
    wb.save(output_file)


#4 Add the Formulas

In [3]:

# Load the workbook
wb = load_workbook(output_file)

# Add formulas to each sheet
for sheet in wb.sheetnames:
    if sheet == 'Sheet':  # Skip the initial blank sheet
        continue
    ws = wb[sheet]
    
    # Add headers
    ws.insert_rows(0)
    ws['A1'], ws['B1'], ws['C1'] = 'datetime', 'scale', 'temperature'
    
    # Find the maximum row number
    max_row = ws.max_row
    
    # Add formulas for min, max, average temperature
    ws['G2'], ws['G3'], ws['G4'] = 'min_temp', 'max_temp', 'mean_temp'
    ws['H2'] = f"=MIN(C2:C{max_row})"
    ws['H3'] = f"=MAX(C2:C{max_row})"
    ws['H4'] = f"=AVERAGE(C2:C{max_row})"
    
    # Add formulas for min, max datetime
    ws['G6'], ws['G7'] = 'Min_Date', 'Max_Date'
    ws['H6'] = f"=MIN(A2:A{max_row})"
    ws['H7'] = f"=MAX(A2:A{max_row})"

    # Check if the style already exists, if not create it
    if 'datetime' not in wb.named_styles:
        date_style = NamedStyle(name='datetime', number_format='YYYY-MM-DD HH:MM:SS')
        wb.add_named_style(date_style)
    
    
    # Apply the style to the min and max datetime cells
    ws['H6'].style = 'datetime'
    ws['H7'].style = 'datetime'

    # Check if the number style already exists, if not create it
    if 'one_decimal' not in wb.named_styles:
        number_style = NamedStyle(name='one_decimal', number_format='0.0')
        wb.add_named_style(number_style)
    
    # Apply the number style to the min, max, and average temperature cells
    ws['H2'].style = 'one_decimal'
    ws['H3'].style = 'one_decimal'
    ws['H4'].style = 'one_decimal'

# Save the workbook with formulas
wb.save(output_file)

Extra Credit

In [4]:
import pandas as pd
from pathlib import Path
from openpyxl import Workbook
from openpyxl.utils.dataframe import dataframe_to_rows

# Define the path to the logs folder
logs_folder = Path("data/logs")

# Define the path to the output folder
output_folder = Path("output")

# Dictionary to hold Workbook objects for each stream identifier
workbooks = {}

# Process each CSV file and insert it into the appropriate Excel workbook
for csv_file in logs_folder.glob("*.csv"):
    # Extract the stream identifier from the file name
    stream_identifier = csv_file.stem.split('-')[0]
    
    # Check if Workbook for this stream already exists, otherwise create a new one
    if stream_identifier not in workbooks:
        workbooks[stream_identifier] = Workbook()
    
    # Read the CSV file into a DataFrame
    df = pd.read_csv(csv_file, header=None, names=['datetime', 'scale', 'temperature'])
    
    # Add a new sheet with the name of the CSV file (without extension)
    sheet_name = csv_file.stem
    ws = workbooks[stream_identifier].create_sheet(title=sheet_name)
    
    # Write the DataFrame to the new sheet
    for row in dataframe_to_rows(df, index=False, header=True):
        ws.append(row)
    
    # Add headers if the sheet is new (not already existing)
    if len(ws['A']) == 1:  # Checking if there is only one row (header row)
        headers = ['datetime', 'scale', 'temperature']
        ws.insert_rows(1)
        for col_num, header in enumerate(headers, start=1):
            ws.cell(row=1, column=col_num, value=header)

# Save each workbook
for stream_identifier, wb in workbooks.items():
    output_file = output_folder / f"{stream_identifier}_consolidated.xlsx"
    wb.save(output_file)
